# Finding whats happening to our failed entries

    First, lets add our old stuff

In [9]:
import pandas as pd
import psycopg2
import numpy as np

df = pd.read_excel(
    "c:/Users/samia/OneDrive/Desktop/ISupply-project/db upload/Map upload/map.xlsx"
)
print(df.head())

   Ser                  combined
0    1               Agriculture
1    2    Agricultural Equipment
2    3  Agricultural Greenhouses
3    4     Aquaculture Equipment
4    5          Aquaculture Trap


In [3]:
def get_category_subcategory(df, index):
    num = df.loc[index, "Ser"]  # O( log(n) )
    name = df.loc[index, "combined"]
    return name, num


def process_categories(df, cursor, con, start, end):
    # listOfCollisions = []
    # listOfAllErrors = []
    queryList = []
    j = 0

    for i in range(start, end):  # O(n)
        name, num = get_category_subcategory(df, i)
        num = str(num)

        query = (
            """INSERT INTO "Categories" (category_id, category) VALUES (%s, %s)""",
            (num, name),
        )
        # print(str(query))
        # query = str(query)
        try:
            cursor.execute(
                """INSERT INTO "Categories" (category_id, category) VALUES (%s, %s)""",
                (num, name),
            )  # O(n)
            # cursor.execute(query)  # O(n)
        except psycopg2.errors.UniqueViolation as e:
            # listOfAllErrors.append(i)
            # listOfCollisions.append(i)
            queryList.append(str(query))
        except Exception as e:
            print(f"Error occurred: {e} q:", query)
            # listOfAllErrors.append(i)
            queryList.append(str(query))

        if j == 10:
            con.commit()
            j = 0  # this time we will commit for each 10th entry
        j = j + 1

    con.commit()

    # print(len(listOfCollisions), len(listOfAllErrors))
    # return listOfCollisions, listOfAllErrors
    print(len(queryList))
    return queryList

    # this should give us some details on where our data is going


    Now lets start using this stuff again

In [4]:

# listOfCollisions = []
# listOfAllErrors = []
queryList = []


# wipe the db from DBeaver

con = psycopg2.connect(
    database="iSupply",
    host="38.180.117.52",
    user="postgres",
    password="deerRun",
    port="5432",
)
cursor = con.cursor()  # open

for i in range(5):  # first 500 for the sake of inspection
    # collisions, errors = process_categories(
    currentQL = process_categories(df, cursor, con, (100 * i), ((100 * i) + 100))
    # listOfCollisions += collisions listOfAllErrors += errors
    queryList += currentQL
    print("100 number ", (i + 1), " done")

# currentQL = process_categories(df, cursor, con, (4500), (4596))
# queryList += currentQL
# print("100 number ", (46), " done")

cursor.close()
con.close()  # close


0
100 number  1  done
1
100 number  2  done
0
100 number  3  done
Error occurred: current transaction is aborted, commands ignored until end of transaction block
 q: ('INSERT INTO "Categories" (category_id, category) VALUES (%s, %s)', ('341', 'Automobiles & Motorcycle s'))
2
100 number  4  done
0
100 number  5  done


In [6]:

print(len(queryList))
queryList

3


['(\'INSERT INTO "Categories" (category_id, category) VALUES (%s, %s)\', (\'151\', nan))',
 '(\'INSERT INTO "Categories" (category_id, category) VALUES (%s, %s)\', (\'340\', nan))',
 '(\'INSERT INTO "Categories" (category_id, category) VALUES (%s, %s)\', (\'341\', \'Automobiles & Motorcycle s\'))']

    Now, lets take a look at the data base

In [10]:
con = psycopg2.connect(
    database="iSupply",
    host="38.180.117.52",
    user="postgres",
    password="deerRun",
    port="5432",
)
cursor = con.cursor()
cursor.execute("""select * from "Categories" c""")
out = cursor.fetchall()  # make a dictionary of what the db currently has
cursor.close()
con.close()

dfDB = pd.DataFrame(out, columns=["ser", "cat"])

dfDB

,ser,cat
0,1,Agriculture
1,2,Agricultural Equipment
2,3,Agricultural Greenhouses
3,4,Aquaculture Equipment
4,5,Aquaculture Trap
...,...,...
475,496,Motorcycle Valve Train
476,497,Other Motorcycle Engines
477,498,Motorcycle Steering
478,499,Motorcycle Tires


    & then lets compare the 2

In [66]:
dfMerged = pd.concat([df.add_suffix('_Original'), dfDB.add_suffix('_DB')], axis=1)
dfMerged = dfMerged.iloc[:500]; dfMerged.head(60)

,Ser_Original,combined_Original,ser_DB,cat_DB
0,1,Agriculture,1.0,Agriculture
1,2,Agricultural Equipment,2.0,Agricultural Equipment
2,3,Agricultural Greenhouses,3.0,Agricultural Greenhouses
3,4,Aquaculture Equipment,4.0,Aquaculture Equipment
4,5,Aquaculture Trap,5.0,Aquaculture Trap
5,6,Fishing Float,6.0,Fishing Float
6,7,Fishing Net,7.0,Fishing Net
7,8,Fishing Rope,8.0,Fishing Rope
8,9,Other Aquaculture Equipment,9.0,Other Aquaculture Equipment
9,10,Ear Tag,10.0,Ear Tag


In [67]:
dfMerged2 = dfMerged.iloc[120:500]; dfMerged2.head(60)

,Ser_Original,combined_Original,ser_DB,cat_DB
120,121,Bonsai,121.0,Bonsai
121,122,Dried Flowers,122.0,Dried Flowers
122,123,Fresh Cut Flowers,123.0,Fresh Cut Flowers
123,124,Natural Plants,124.0,Natural Plants
124,125,Other Agriculture Products,125.0,Other Agriculture Products
125,126,Plant & Animal Oil,126.0,Plant & Animal Oil
126,127,Animal Oil,127.0,Animal Oil
127,128,Blended Oil,128.0,Blended Oil
128,129,Camellia Oil,129.0,Camellia Oil
129,130,Castor Oil,130.0,Castor Oil


    Now lets try making a list of entries not going through

In [68]:
serDbArray = dfMerged['ser_DB'].to_numpy(); serDbArray

array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,
        12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,
        23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,
        34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,  44.,
        45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.,
        56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,  66.,
        67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,  77.,
        78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,
        89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,  99.,
       100., 101., 102., 103., 104., 105., 106., 107., 108., 109., 110.,
       111., 112., 113., 114., 115., 116., 117., 118., 119., 120., 121.,
       122., 123., 124., 125., 126., 127., 128., 129., 130., 131., 132.,
       133., 134., 135., 136., 137., 138., 139., 140., 141., 152., 153.,
       154., 155., 156., 157., 158., 159., 160., 16